In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds

In [2]:
df, df_info = tfds.load("food101", as_supervised=True, shuffle_files=True, with_info=True, download=True)

In [3]:
train_df, valid_df = df["train"], df["validation"]

In [4]:
valid_df = valid_df.map(lambda image, label: (tf.cast(tf.image.resize(image, [50,50]), dtype=tf.int32), tf.cast(label == 55, dtype=tf.int32)))

In [5]:
train_df = train_df.map(lambda image, label: (tf.cast(tf.image.resize(image, [50, 50]), dtype=tf.int32), tf.cast(label == 55, dtype=tf.int32)))

In [6]:
hotdogs = train_df.filter(lambda image, label: label == 1).repeat(3)
not_hotdogs = train_df.filter(lambda image, label: label == 0)

In [7]:
valid_hotdogs = valid_df.filter(lambda image, label: label == 1).repeat(3)
valid_not_hotdogs = valid_df.filter(lambda image, label: label == 0)

In [8]:
train_ds = tf.data.Dataset.sample_from_datasets([hotdogs, not_hotdogs], [0.5, 0.5], stop_on_empty_dataset=True)
train_ds = train_ds.cache().batch(16).prefetch(tf.data.AUTOTUNE)

In [9]:
valid_ds = tf.data.Dataset.sample_from_datasets([valid_hotdogs, valid_not_hotdogs], [0.5, 0.5], stop_on_empty_dataset=True)
valid_ds = valid_ds.cache().batch(16).prefetch(tf.data.AUTOTUNE)

In [16]:
data_augmentation = models.Sequential([layers.RandomFlip("horizontal"), layers.RandomRotation(0.2)])

In [23]:
model = models.Sequential()
model.add(layers.Rescaling(1./255))
model.add(data_augmentation)
model.add(layers.Conv2D(128, (3, 3), activation="relu", input_shape=[50, 50, 3]))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1))

In [24]:
lr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [25]:
history = model.fit(train_ds, validation_data=valid_ds, epochs=40, verbose=1)

Epoch 1/40
281/281 [==============================] - 27s 83ms/step - loss: 1.2175 - accuracy: 0.4986 - val_loss: 0.9803 - val_accuracy: 0.5295
Epoch 2/40
281/281 [==============================] - 22s 78ms/step - loss: 0.8709 - accuracy: 0.5035 - val_loss: 0.7927 - val_accuracy: 0.5307
Epoch 3/40
281/281 [==============================] - 23s 81ms/step - loss: 0.7586 - accuracy: 0.5329 - val_loss: 0.7225 - val_accuracy: 0.5828
Epoch 4/40
281/281 [==============================] - 23s 80ms/step - loss: 0.7168 - accuracy: 0.5376 - val_loss: 0.6992 - val_accuracy: 0.5866
Epoch 5/40
281/281 [==============================] - 25s 87ms/step - loss: 0.6939 - accuracy: 0.5587 - val_loss: 0.6792 - val_accuracy: 0.5922
Epoch 6/40
281/281 [==============================] - 24s 85ms/step - loss: 0.6847 - accuracy: 0.5545 - val_loss: 0.6614 - val_accuracy: 0.6173
Epoch 7/40
281/281 [==============================] - 23s 82ms/step - loss: 0.6696 - accuracy: 0.5761 - val_loss: 0.6513 - val_accuracy: